# Generate mesh of well conformers
This uses a RDkit as backend to generate the conformer and calculate the energy by Psi4.

In [ ]:
import os
import multiprocessing
from itertools import product
from typing import List, Optional, Union

import numpy as np
import psi4
import pybel
from joblib import Parallel, delayed
from rdkit import Chem

from ccg.backend.rdk import RDKitMol, RDKitConf
from ccg.converter.geom import (xyz_str_to_xyz_dict,
                                xyz_dict_to_xyz_str,
                                xyz_dict_to_xyz_file)

# HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def get_separable_angle_list(samplings: Union[list, tuple],
                             original_angles: Union[list, tuple]):
    """
    Get a angle list for each input dimension. For each dimension
    The input can be a int, indicating the angles will be evenly sampled;
    Or a list, indicate the angles to be sampled;
    Examples:
    [[120, 240,], 4, 0] => [np.array([120, 240,]), 
                            np.array([0, 90, 180, 270,]), 
                            np.array([0])]
    List of np.arrays are returned for the sake of further calculation
    
    Args:
        samplings (Union[list, tuple]): An array of sampling information.
                  For each element, it can be either list or int.
    
    Returns:
        list: A list of sampled angles sets.
    """
    original_angles = original_angles or len(samplings) * [0.]
    angle_list = []
    for ind, angles in enumerate(samplings):
        # Only provide a number
        # This is the step number of the angles
        if isinstance(angles, (int, float)):
            try:
                step = 360 // angles
            except ZeroDivisionError:
                # Does not change
                angles = original_angles[ind] + np.array([0])
            else:
                angles = original_angles[ind] + \
                         np.array([step * i for i in range(angles)])
        elif isinstance(angles, list):
            angles = original_angles[ind] + np.array(angles)
        angle_list.append(angles.tolist())
            
    return angle_list


def get_angle_mesh(angle_list: list):
    """
    Get the angle mesh. The created mesh is a Cartesian product of the list.
    
    Args:
        angle_list (list): A list of angle list.
    """
    return product(*angle_list)
    

def conformers_by_change_torsions(conf: 'RDKitConf',
                                  angle_mesh,
                                  bookkeep: dict,
                                  torsions=None,
                                  on_the_fly_check=False):
    """
    Generate conformers by rotating the angles of the torsions. The result will be saved into
    ``bookkeep``. A on-the-fly check can be applied, which identifies the conformers with colliding
    atoms.
    
    Args:
        conf (RDkitConf): A RDKit Conformer to be used.
        angle_mesh (iterable): An iterable contains the angle_list for conformers to be generated from.
        bookkeep (dict): A dictionary to save the coords.
        torsions (list): A list of four-atom-index lists indicating the torsional modes.
        on_the_fly_filter (bool): Whether to check colliding atoms on the fly.
    """
    if not torsions:
        torsions = conf.GetTorsionalModes()
    for ind, angles in enumerate(angle_mesh):
        
        conf.SetAllTorsionsDeg(angles)
        bookkeep[ind] = {'angles': angles,
                         'coords': conf.GetPositions().tolist()}
        bookkeep[ind]['colliding_atoms'] = conf.HasCollidingAtoms() \
                if on_the_fly_check == True else None

## 1.0 Perceive a molecule

In [ ]:
xyz_str="""O       1.85073500   -1.14075800   -0.80808500
O       1.64413700   -0.98958800    0.61920900
C       0.89993200    0.19167500    0.83332300
C       1.57997200    1.46553500    0.32458600
C       0.87059800    2.73251700    0.76632100
O       2.94131500    1.54377500    0.86706900
O       3.78534700    0.75777800    0.22699800
H       2.74479300   -0.76185300   -0.87545700
H       0.78501500    0.22753800    1.92086600
H      -0.09098800    0.11998200    0.36665200
H       1.69085200    1.41865800   -0.75897600
H       0.84097800    2.80358600    1.85617500
H      -0.15579100    2.73441900    0.39232200
H       1.37972400    3.61444100    0.37512000"""

In [ ]:
xyz_file = f"{len(xyz_str.splitlines())}\n\n{xyz_str}"
xyz_dict = xyz_str_to_xyz_dict(xyz_str)
pybel_mol = pybel.readstring('xyz', xyz_file)

## 1.1 Use RDKit to generate conformers

In [ ]:
rdkitmol = RDKitMol.FromOBMol(pybel_mol.OBMol)

rdkitmol.EmbedMultipleConfs(1)
confs = rdkitmol.GetAllConformers()
confs[0].SetPositions(xyz_dict['coords'])

torsions = rdkitmol.GetTorsionalModes()
num_torsions = len(torsions)
original_angles = confs[0].GetAllTorsionsDeg()

angles_list = get_separable_angle_list([3]*5,
                                       original_angles)
angle_mesh = product(*angles_list)

bookkeep = {}
conformers_by_change_torsions(confs[0],
                              angle_mesh,
                              bookkeep=bookkeep,
                              on_the_fly_check=True)

### [OPTIONAL] Print Conformers with colliding atoms

In [ ]:
num = 0
for value in bookkeep.values():
    if value['colliding_atoms']:
        print(f'{value}\n')
        num += 1
print(f"\n{num} out of {len(bookkeep)} conformers may have colliding atoms")

### [OPTIONAL] Align Conformers

In [ ]:
import py3Dmol

In [ ]:
rdkitmol.EmbedMultipleConfs(len(bookkeep))
confs = rdkitmol.GetAllConformers()
torsions_values = []
for i, value in bookkeep.items():
    confs[i].SetPositions(value['coords'])
    torsion_value = np.array(confs[i].GetAllTorsionsDeg())
    for tv in torsions_values:
        if np.allclose(tv, torsion_value):
            print(i)
            break
    else:
        torsions_values.append(torsion_value)
rd_mol = rdkitmol.ToRDMol()

In [ ]:
rmsd_list = []
Chem.rdMolAlign.AlignMolConformers(rd_mol, maxIters=50, RMSlist=rmsd_list)
# rmsd_list 

In [ ]:
view = py3Dmol.view(width=600, height=400)
for i in range(len(bookkeep)):
    view.addModel(Chem.MolToMolBlock(rd_mol, confId=i), 'sdf')
view.setStyle({'stick':{'colorscheme':'cyanCarbon'}})
view.render()

## 2. Calculate Psi4

In [ ]:
# use uhf for open shell species and rhf for closed shell species!
def get_psi4_dftenergy(geom):
    psi4.set_memory('8000 MB')
    psi4.set_options({'reference': 'uhf'})
    try:
        psi4.geometry(geom)
        return psi4.energy('wb97x-d/def2-svp')
    except:
        return np.inf

import time

def wrapper(i):
    start = time.time()
    xyz_dict['coords'] = bookkeep[i]['geom']
    xyz_file = xyz_dict_to_xyz_file(xyz_dict)
    value['energy'] = get_psi4_dftenergy(xyz_file)
    end = time.time()
    print(f"Conformer {i}, time spent: {end - start}")

Parallel(n_jobs=4)(delayed(wrapper)(i) for i in range(len(bookkeep)))